## testing testing. is anybody in there?

In [ ]:
# eval?

In [1]:
from datasets import Dataset, load_dataset
from pandas import *
# fixing the way we read from the excel files
def get_manipulation_questions(path = 'data/manipulation_tasks.xlsx'):
    # makes the initial Dataset
    xls = ExcelFile(path)
    df = xls.parse(xls.sheet_names[0])
    questions = df['Text Question'].to_list()
    rcg = df['red_cube_goal'].to_list()
    bcg = df['blue_cube_goal'].to_list()
    ycg = df['yellow_cube_goal'].to_list()
    gcg = df['green_cube_goal'].to_list()
    answers = []
    for i in range(len(rcg)):
        answers.append({'red_cube_goal':eval(rcg[i]),
                        'blue_cube_goal':bcg[i],
                        'yellow_cube_goal':ycg[i],
                        'green_cube_goal':gcg[i]})
    data = Dataset.from_dict({'question':questions,'answer':answers})

    # does some other fucky thing idk
    data = data.map(
        lambda x: {  # type: ignore
            "prompt": [
                {"role": "system", "content": 'SYSTEM_PROMPT'},
                {"role": "user", "content": x["question"]},
            ],
            "answer":(x["answer"]),
        }
    )  # type: ignore
    return data  # type: ignore
data = get_manipulation_questions()

c:\Users\qtcc\miniconda3\envs\genesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 4/4 [00:00<00:00, 1000.43 examples/s]


In [2]:
print(data[0]['answer'])

{'blue_cube_goal': '(-0.25,0.25,0.02)', 'green_cube_goal': '(-0.25,0.5,0.02)', 'red_cube_goal': [0.0, 0.375, 0.02], 'yellow_cube_goal': '(0.25,0.5,0.02)'}


In [1]:
# something about the processing
thing = '<answer>\nmove_x(0.25)\nmove_y(0.25)\nmove_z(0.02)\ngripper_close()\nmove_x(0)\nmove_y(0.375)\nmove_z(0.02)\nmove_z(-0.4)\ngripper_open()\nmove_z(0)\n</answer>'
def find_between(s, start, end):
    plan = ''
    try:
        plan = s.split(start)[1].split(end)[0]
    except:
        plan = ''
    return plan
another_thing = find_between(thing,'<answer>','</answer>')
for line in another_thing.splitlines():
    print(line)


move_x(0.25)
move_y(0.25)
move_z(0.02)
gripper_close()
move_x(0)
move_y(0.375)
move_z(0.02)
move_z(-0.4)
gripper_open()
move_z(0)


In [2]:
# meow
thing = '<answer>\nmove_x(0.25)\nmove_y(0.125)\nmove_z(0.02)\ngripper_close()\nmove_z(0.48)  # Adjust the z-position to move closer to the target\nmove_x(0)\nmove_y(0.375)  # Adjust as needed to reach the y-axis target\ngripper_open()\nmove_z(0.02)  # Adjust the z-position back to avoid collisions\n</answer>'
def find_between(s, start, end):
    plan = ''
    try:
        plan = s.split(start)[1].split(end)[0]
    except:
        plan = ''
    return plan
another_thing = find_between(thing,'<answer>','</answer>')
for line in another_thing.splitlines():
    print(line)


move_x(0.25)
move_y(0.125)
move_z(0.02)
gripper_close()
move_z(0.48)  # Adjust the z-position to move closer to the target
move_x(0)
move_y(0.375)  # Adjust as needed to reach the y-axis target
gripper_open()
move_z(0.02)  # Adjust the z-position back to avoid collisions


In [25]:
# getting our dataset format to work the same 
from pandas import *
from datasets import Dataset

with open("prompts/base_manip_prompt.txt","r") as f:
    SYSTEM_PROMPT = f.read()
def get_manipulation_questions(path = 'data/manipulation_tasks.xlsx'):
    # makes the initial Dataset
    xls = ExcelFile(path)
    df = xls.parse(xls.sheet_names[0])
    questions = df['Text Question'].to_list()
    rcg = df['red_cube_goal'].to_list()
    bcg = df['blue_cube_goal'].to_list()
    ycg = df['yellow_cube_goal'].to_list()
    gcg = df['green_cube_goal'].to_list()
    answers = []
    for i in range(len(rcg)):
        answers.append({'red_cube_goal':rcg[i],
                        'blue_cube_goal':bcg[i],
                        'yellow_cube_goal':ycg[i],
                        'green_cube_goal':gcg[i]})
    data = Dataset.from_dict({'question':questions,'answer':answers})

    # does some other fucky thing idk
    data = data.map(
        lambda x: {  # type: ignore
            "prompt": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": x["question"]},
            ],
            "answer":(x["answer"]),
        }
    )  # type: ignore
    return data  # type: ignore
dataset = get_manipulation_questions()
dataset[0]

Map: 100%|██████████| 4/4 [00:00<00:00, 1333.11 examples/s]


{'question': 'Take the block with a colour on the Canadian flag, and move it to the position (0,0.375,0.02). ',
 'answer': {'blue_cube_goal': '(-0.25,0.25,0,0.02)',
  'green_cube_goal': '(-0.25,0.5,0,0.02)',
  'red_cube_goal': '(0,0.375,0.02)',
  'yellow_cube_goal': '(0.25,0.5,0,0.02)'},
 'prompt': [{'content': 'You are a helpful assistant that will help me control a robotic manipulator to accomplish specific tasks. We have a setup including a basic robotic arm bolted to a table. On the table is a set of four coloured blocks (red, blue, yellow, and green) that I will want you to help move around. \n\nI will give you a specific task to accomplish, along with a list of action primitives that you have the ability to execute.\n\nEach action primitive is specified by a Python-like function header. I will describe each one and what they do in detail below. Some primitives also have a parameter, which specifies the degree to which that primitive should be executed.  \n\nThe robotic arm you wi

In [2]:
# stupid import shit i gotta test
from genesis_env import FrankaManipEnv
